In [129]:
import yfinance as yf
import talib
import pandas as pd
import numpy as np
import vectorbt as vbt 
from vectorbt.portfolio.enums import OrderSide

# data preprocess
target_stock = "1605"
threshold = 0
stock_df = pd.read_csv(r'C:\Users\Admin\Desktop\量化交易\lstm 預測股價(squeeze中包含的技術分析)\LSTM_stock_predicting\data\1605 2023.csv', encoding='utf-8', encoding_errors= 'ignore')
five_day_min_low = talib.MIN(stock_df['Low'], timeperiod=5) # 五天最低價當作long strategy的停損點
five_day_max_high = talib.MAX(stock_df['High'], timeperiod=5) # 五天最高價當作short strategy的停損點
stock_df['five_day_min_low'] = five_day_min_low
stock_df['five_day_max_high'] = five_day_max_high
stock_df = stock_df[25:-1]
predicted_price_df = pd.read_csv(r"C:\Users\Admin\Desktop\量化交易\lstm 預測股價(squeeze中包含的技術分析)\LSTM_stock_predicting\predicted_result\GRUModel 1605 2023 predicted prices.csv", encoding='utf-8', encoding_errors = 'ignore')
stock_df['Date'] = pd.to_datetime(stock_df['Date'])
stock_df.index = stock_df['Date']
stock_df['predicted_price'] = predicted_price_df['predicted_price'].values
stock_df['true_price'] = predicted_price_df['true_price'].values

# long short condition
stock_df['predicted_price_up'] = (stock_df['predicted_price'] > stock_df['predicted_price'].shift(1)).astype(int)
stock_df['actual_price_up'] = (stock_df['Close'] > stock_df['Close'].shift(1)).astype(int)
stock_df['long_condition'] = (stock_df['true_price'].shift(1)<=stock_df['predicted_price']) & ((stock_df['predicted_price']-stock_df['true_price'].shift(1))/stock_df['true_price'].shift(1) >= threshold)
stock_df['short_condition'] = (stock_df['true_price'].shift(1)>stock_df['predicted_price']) & (abs((stock_df['predicted_price']-stock_df['true_price'].shift(1))/stock_df['true_price'].shift(1)) >= threshold)
stock_df['long_condition'] = stock_df['long_condition'].astype(int)
stock_df['short_condition'] = stock_df['short_condition'].astype(int)

stock_df

,Date,Capacity,Turnover,Open,High,Low,Close,Change,Transcation,buy_Dealer_Hedging,...,sell_Foreign_Investor,sell_Investment_Trust,five_day_min_low,five_day_max_high,predicted_price,true_price,predicted_price_up,actual_price_up,long_condition,short_condition
Date,,,,,,,,,,,,,,,,,,,,,
2023-02-17,2023-02-17,102368880,5727494455,55.5,57.90,54.50,57.50,1.20,42029,1933159,...,23096656,10402,52.30,57.90,1.014947,1.201960,0,0,0,0
2023-02-20,2023-02-20,68083817,3978205083,57.6,59.30,57.20,58.60,1.10,30746,422500,...,12895000,150000,52.80,59.30,1.028901,1.228250,1,1,0,1
2023-02-21,2023-02-21,63545032,3708739948,58.6,59.40,57.50,57.80,-0.80,26183,217465,...,11254885,100000,52.80,59.40,1.046954,1.209130,1,0,0,1
2023-02-22,2023-02-22,49757636,2855899655,57.5,58.10,56.70,57.30,-0.50,22647,114000,...,14577263,305216,54.30,59.40,1.066004,1.197180,1,0,0,1
2023-02-23,2023-02-23,34817747,2007386642,57.5,58.20,57.30,57.30,0.00,15462,144454,...,10309000,209000,54.50,59.40,1.077695,1.197180,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22,2023-12-22,31609461,1241329965,39.4,39.65,38.85,38.85,-0.70,12800,114841,...,14851000,25825,38.20,39.65,0.717604,0.756214,1,0,0,1
2023-12-25,2023-12-25,16348214,630348117,39.3,39.30,38.15,38.15,-0.70,8380,22000,...,4400000,1675,38.15,39.65,0.716879,0.739484,0,0,0,1
2023-12-26,2023-12-26,11133245,425951999,38.5,38.55,38.15,38.20,0.05,5409,31000,...,2820000,1304675,38.15,39.65,0.714233,0.740679,0,1,0,1


strategy


In [130]:
# long strategy
signals_long = np.zeros(len(stock_df))
partition_size_long = 0
stop_loss_long = 0
for i in range(len(stock_df)):
    if partition_size_long == 0:
        if stock_df['long_condition'][i] == 1:
            signals_long[i] = 1
            partition_size_long = 1
            stop_loss_long = stock_df['five_day_min_low'][i]
    if partition_size_long == 1:
        if (stock_df['Close'][i] <= stop_loss_long) or (stock_df['short_condition'][i] == 1): # 停利停損
            signals_long[i] = -1
            stop_loss_long = 0 # 停損點重置
            partition_size_long = 0 # 平倉

# short strategy
signals_short = np.zeros(len(stock_df))
partition_size_short = 0
stop_loss_short = 0
for i in range(len(stock_df)):
    if partition_size_short == 0:
        if stock_df['short_condition'][i] == 1:
            signals_short[i] = -1
            partition_size_short = -1
            stop_loss_short = stock_df['five_day_max_high'][i]
    if partition_size_short == -1:
        if (stock_df['Close'][i] >= stop_loss_short) or (stock_df['long_condition'][i] == 1):# 停利停損
            signals_short[i] = 1
            stop_loss_short = 0 # 停損點重置
            partition_size_short = 0 # 平倉

entries_long = signals_long == 1
exits_long = signals_long == -1
entries_short = signals_short == -1
exits_short = signals_short == 1
pf = vbt.Portfolio.from_signals(stock_df['Open'], 
                                    entries=entries_long,
                                    exits=exits_long, 
                                    short_entries=entries_short,
                                    short_exits=exits_short,
                                    fees = 0,
                                    freq='1D',
                                    direction='both',)
# print(pf_long.stats().to_string()) # to_string()可以將全部結果攤開
print(pf.stats().to_string())
'''entries_short = signals_short == -1
exits_short = signals_short == 1
pf_short = vbt.Portfolio.from_signals(data['Close'], entries_short, exits_short, direction='shortonly')'''
# print(pf_short.stats().to_string())

Start                               2023-02-17 00:00:00
End                                 2023-12-28 00:00:00
Period                                213 days 00:00:00
Start Value                                       100.0
End Value                                    200.919593
Total Return [%]                             100.919593
Benchmark Return [%]                         -29.009009
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                               6.486486
Max Drawdown Duration                  23 days 00:00:00
Total Trades                                         23
Total Closed Trades                                  22
Total Open Trades                                     1
Open Trade PnL                               -13.936619
Win Rate [%]                                  77.272727
Best Trade [%]                                12.553191
Worst Trade [%]                               -4

C:\Users\Admin\AppData\Local\Temp\ipykernel_24388\2404176544.py:7: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\Admin\AppData\Local\Temp\ipykernel_24388\2404176544.py:10: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\Admin\AppData\Local\Temp\ipykernel_24388\2404176544.py:12: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\Admin\AppData\Local\Temp\ipykernel_24388\2404176544.py:23: FutureWarning:

Series.__ge

"entries_short = signals_short == -1\nexits_short = signals_short == 1\npf_short = vbt.Portfolio.from_signals(data['Close'], entries_short, exits_short, direction='shortonly')"

visualization


In [131]:
pf.plot().show()